In [1]:
# Preparing Data
import pandas as pd
import featuretools as ft
# Load the rock mines dataset
url = "https://archive.ics.uci.edu/ml/machine-learning-databases/undocumented/connectionist-bench/sonar/sonar.all-data"
df = pd.read_csv(url)
column_names = ["sensor_" + str(i) for i in range(1, 61)] + ["target"]
df.columns = column_names
df = df.reset_index()
df = df.rename(columns={"index": "id"})
df['target'] = df['target'].map({'M': 1, 'R': 0})
x_data = df.iloc[:, :60]
x_data = x_data.drop('id', axis=1)
y_data = df['target']

##### Get Base Scoring
from sklearn.metrics import log_loss
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import*
kfold = StratifiedKFold(n_splits=10, random_state=None)
model = RandomForestClassifier()
scoring = 'neg_log_loss'
results = cross_val_score(model, x_data, y_data, cv=kfold, scoring=scoring)
print(results.mean())






-0.524814091460936


In [2]:
##### RCEV for automatic Feature Selection

from sklearn.feature_selection import RFECV
from sklearn.ensemble import RandomForestClassifier
# Apply
model_RFE = RandomForestClassifier()
#step= how many features are eliminated each iteration
#step 0-1 how much percent is eliminated
kfold = StratifiedKFold(n_splits=3, random_state=None)
rfecv = RFECV(estimator=model_RFE, step=1, cv=kfold, scoring='neg_log_loss')
rfecv.fit(x_data, y_data)
selected_features = pd.DataFrame({
    'Feature': x_data.columns,
    'Selected': rfecv.support_,
    'Rank': rfecv.ranking_
})
selected_columns = selected_features[selected_features['Selected']]['Feature'].tolist()
x_data_RFE = x_data[selected_columns]
print(len(selected_columns))


kfold = StratifiedKFold(n_splits=10, random_state=None)
model = RandomForestClassifier()
scoring = 'neg_log_loss'
results = cross_val_score(model, x_data_RFE, y_data, cv=kfold, scoring=scoring)
print(results.mean())

52
-0.5100185860679436


In [3]:
##### RFE Finetuning, manual

from sklearn.feature_selection import RFE
model_RFE = RandomForestClassifier()
rfe = RFE(estimator=model_RFE, n_features_to_select=25)
rfe.fit(x_data, y_data)
feature_names = x_data.columns[rfe.support_]
x_data_RFE_fine= x_data[feature_names]
kfold = StratifiedKFold(n_splits=10, random_state=None)
#odel = RandomForestClassifier()
scoring = 'neg_log_loss'
results = cross_val_score(model, x_data_RFE_fine, y_data, cv=kfold, scoring=scoring)
print(results.mean())


-0.4777705567956114
